### Demonstrate the idea of a spread envelope

See my notes

It is defined as a function of s given by
$$
\cal{E}_{f,g}(s) = \max_{x-y=s} \{  f(x) + g(y) \}
$$

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import cvxpy as cp
from matplotlib import cm
import matplotlib.pyplot as plt

from sprd import grids, opt

In [3]:
np.set_printoptions(suppress=True)
np.set_printoptions(precision = 4)
%matplotlib widget

In [4]:
# do we want to save figures for notes/slides
save_figures = False

Set up grids for X,Y and X-Y

In [5]:

# set up the grid for X and Y
npts = 101
fwd_for_grid = 0.0
std_for_grid = 1.0
n_std = 3
xgrid = grids.gen_grid(npts, fwd = fwd_for_grid, stdev=std_for_grid, nstd = n_std)

# this does assume equidistant xgrid -- but we check
assert (np.linalg.norm(np.diff(xgrid, 2)) < 1e-6)

# set up the grid for x-y
sgrid = np.concatenate([xgrid - xgrid[-1], xgrid[1:] - xgrid[0]])

# as 2D
sx,sy = np.meshgrid(xgrid,xgrid,indexing = 'ij')

### An example of a spread envelope calculation for fixed f,g

In [6]:
f = 1.0*(xgrid > 0)
g = 1.0*(xgrid > 0)

sf, sg = np.meshgrid(f,g,indexing = 'ij')
fpg = sf + sg

env = np.zeros_like(sgrid)

for si in range(len(sgrid)):

    # the right diagonal
    dg = np.diag(fpg, si - npts+1)
    env[si] = np.max(dg)

# print(env)



In [7]:
def spread_envelope(f : np.ndarray, g: np.ndarray): 
    '''
    A function that takes f(x) and g(y) as 1D arrays with npts elements
    and returns h(s), their spread envelope, as a a 1D array with 2*npts-1 
    elements, as well a a 2D version
    '''
    sf, sg = np.meshgrid(f,g,indexing = 'ij')
    fpg = sf + sg

    env1D = np.zeros_like(sgrid)
    env2D = np.zeros_like(fpg)

    for si in range(len(sgrid)):

        # the  diagonal s = x-y
        dg = np.diag(fpg, si - npts+1)

        # find the max on that diagonal of f(x) + g(y). This is env(s)
        env1D[si] = np.max(dg)

        # create a 2D version
        env_curr_diag = np.diag(np.ones_like(dg)*env1D[si], si - npts + 1)
        env2D += env_curr_diag


    return env1D, env2D, fpg

### set f,g and plot

In [8]:
# set f and g to whatever you want ...

# e.g. digitals...
f = 1.0*(xgrid > 0)
g = 1.0*(xgrid < 0)

# or calls/puts ...
# f = np.maximum(xgrid,0)
# g = np.maximum(-xgrid, 0)

# ... and calculate the spread envelope
h,sh,fpg = spread_envelope(f,g)


### Line view of the spread envelope

In [ ]:
plt.figure()
plt.plot(sgrid, h, label = 'Spread envelope $E_{f,g}(s)$')
plt.legend(loc='best')
plt.show()

### Surface view of the spread envelope

In [ ]:
step = 3

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(sx[::step,::step], sy[::step,::step], sh[::step,::step], marker = '.', alpha = 0.5, label = 'spread envelope')
surf = ax.plot_surface(sx[::step,::step], sy[::step,::step], fpg[::step,::step], cmap = 'Reds', alpha=0.7, label = 'f+g')

plt.xlabel('x')
plt.ylabel('y')
plt.title('$f(x)+g(y)$ and its spread envelope $E_{f,g}(x-y)$')

# some magic workaround for the legend() bug, see 
# https://stackoverflow.com/a/54994985/14551426
# does not seem to work
surf._facecolors2d = surf._facecolor3d
surf._edgecolors2d = surf._edgecolor3d

plt.legend(loc = 'best')
plt.show()
if save_figures:
    fname = './figs/spread_envelope_01.pdf'
    fig.savefig(fname, bbox_inches='tight')